In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

# Functions

In [2]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [3]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,method
0,0.097175,exp1,Sphere,1.0,NN_HMu
1,0.096994,exp1,Sphere,1.0,NN_HMu
2,0.096883,exp1,Sphere,1.0,NN_HMu
3,0.096573,exp1,Sphere,1.0,NN_HMu
4,0.096567,exp1,Sphere,1.0,NN_HMu


In [4]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])

In [5]:
print(table2.to_csv(float_format='%.2f'))

experiment,,exp1,exp1,exp1,exp1,exp1,exp2,exp2,exp2,exp2,exp2,exp3,exp3,exp3,exp3,exp3,exp4,exp4,exp4,exp4,exp4
freq,,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0
function,method,,,,,,,,,,,,,,,,,,,,
Rastrigin,NN_CwN,0.19,0.10,0.04,0.01,0.00,0.20,0.10,0.04,0.01,0.00,0.20,0.10,0.04,0.01,0.00,0.19,0.09,0.04,0.01,0.00
Rastrigin,NN_HMu,0.19,0.10,0.02,0.01,0.01,0.19,0.10,0.02,0.01,0.00,0.20,0.12,0.02,0.01,0.01,0.19,0.10,0.02,0.01,0.00
Rastrigin,NN_No,0.20,0.10,0.02,0.01,0.00,0.20,0.12,0.02,0.01,0.00,0.20,0.10,0.03,0.01,0.00,0.19,0.09,0.02,0.01,0.00
Rastrigin,NN_RI,0.20,0.10,0.02,0.01,0.00,0.22,0.10,0.02,0.01,0.01,0.20,0.12,0.02,0.01,0.01,0.20,0.10,0.02,0.01,0.00
Rosenbrock,NN_CwN,0.19,0.10,0.04,0.01,0.00,0.19,0.10,0.04,0.01,0.00,0.20,0.10,0.04,0.01,0.01,0.19,0.10,0.04,0.01,0.01
Rosenbrock,NN_HMu,0.19,0.10,0.02,0.01,0.00,0.20,0.10,0.02,0.01,0.00,0.20,0.10,0.02,0.01,0.00,0.19,0.12,0.02,0.01,0.00
Rosenbrock,NN_No,0.20,0.10,0.02,0.01,0.00,0.19,0.10,0.02

In [6]:
table2

experiment             exp1                                              exp2  \
freq                   0.5       1.0       4.0       10.0      20.0      0.5    
function   method                                                               
Rastrigin  NN_CwN  0.194483  0.097421  0.039736  0.009750  0.004897  0.199157   
           NN_HMu  0.193391  0.098932  0.024560  0.009822  0.005953  0.193947   
           NN_No   0.196051  0.099062  0.024410  0.009854  0.004973  0.195018   
           NN_RI   0.197259  0.098183  0.024361  0.009991  0.004952  0.221761   
Rosenbrock NN_CwN  0.194411  0.097156  0.039955  0.009685  0.004908  0.193652   
           NN_HMu  0.193332  0.097850  0.024838  0.009848  0.004965  0.195056   
           NN_No   0.197486  0.096047  0.024298  0.009801  0.004925  0.194123   
           NN_RI   0.194902  0.099433  0.024556  0.009724  0.004883  0.220409   
Sphere     NN_CwN  0.227201  0.097152  0.039943  0.009686  0.004883  0.195255   
           NN_HMu  0.196841  0.098117  0.024857  0.009899  0.004961  0.193648   
           NN_No   0.198567  0.096965  0.024232  0.009868  0.004947  0.195548   
           NN_RI   0.231917  0.099392  0.024666  0.009974  0.004917  0.197886   

experiment                                                     exp3            \
freq                   1.0       4.0       10.0      20.0      0.5       1.0    
function   method                                                               
Rastrigin  NN_CwN  0.098715  0.040646  0.009949  0.004977  0.199107  0.098870   
           NN_HMu  0.098371  0.024591  0.009872  0.004979  0.203568  0.119373   
           NN_No   0.119375  0.024577  0.009992  0.004912  0.198645  0.098791   
           NN_RI   0.098489  0.024573  0.009880  0.005040  0.199399  0.118550   
Rosenbrock NN_CwN  0.097336  0.040052  0.009762  0.004836  0.199657  0.100296   
           NN_HMu  0.098521  0.024627  0.011584  0.004906  0.199310  0.100012   
           NN_No   0.099290  0.024468  0.009921  0.004885  0.199095  0.102680   
           NN_RI   0.098045  0.028848  0.009854  0.005013  0.200148  0.099863   
Sphere     NN_CwN  0.096794  0.039883  0.009797  0.004853  0.234993  0.100299   
           NN_HMu  0.098405  0.024580  0.009815  0.004893  0.199823  0.100005   
           NN_No   0.097277  0.029127  0.009943  0.005003  0.198499  0.100723   
           NN_RI   0.098748  0.024444  0.009945  0.004929  0.197989  0.100184   

experiment                                           exp4                      \
freq                   4.0       10.0      20.0      0.5       1.0       4.0    
function   method                                                               
Rastrigin  NN_CwN  0.040898  0.010018  0.004975  0.189538  0.094374  0.039017   
           NN_HMu  0.024917  0.009884  0.005984  0.193388  0.098800  0.023808   
           NN_No   0.025038  0.009870  0.004937  0.192611  0.094863  0.024017   
           NN_RI   0.024976  0.010036  0.005038  0.195949  0.097258  0.024252   
Rosenbrock NN_CwN  0.040931  0.010047  0.005008  0.191358  0.095137  0.039425   
           NN_HMu  0.024859  0.009858  0.004972  0.193739  0.115593  0.024556   
           NN_No   0.024790  0.009899  0.004959  0.193144  0.094907  0.024963   
           NN_RI   0.024720  0.009933  0.005052  0.195907  0.096677  0.024758   
Sphere     NN_CwN  0.040340  0.009983  0.005001  0.189387  0.094701  0.039698   
           NN_HMu  0.024837  0.009977  0.004970  0.194313  0.097102  0.025189   
           NN_No   0.024997  0.010004  0.005048  0.192740  0.096577  0.024982   
           NN_RI   0.024625  0.009949  0.005011  0.190290  0.096834  0.024815   

experiment                             
freq                   10.0      20.0  
function   method                      
Rastrigin  NN_CwN  0.009363  0.004819  
           NN_HMu  0.009579  0.004988  
           NN_No   0.009582  0.004838  
           NN_RI   0.009624  0.004933  
Rosenbrock NN_CwN  0.009526  0.005677  
           NN_HMu  0.009841  0.0